In [1]:
# PyTorch的核心是两个主要特征：

# 1） 一个n维张量，类似于numpy，但可以在GPU上运行
# 2） 搭建和训练神经网络时的自动微分/求导机制
# 本章节我们将使用全连接的ReLU网络作为运行示例。该网络将有一个单一的隐藏层，并将使用梯度下降训练，
# 通过最小化网络输出和真正 结果的欧几里得距离，来拟合随机生成的数据

In [9]:
# 1 张量

# 热身: Numpy
# 在介绍PyTorch之前，本章节将首先使用numpy实现网络。 Numpy提供了一个n维数组对象，
# 以及许多用于操作这些数组的 函数。Numpy是用于科学计算的通用框架;它对计算图、深度学习和梯度一无所知。
# 然而，我们可以很容易地使用NumPy，手动实现网络的 前向和反向传播，来拟合随机数据：

# -*- coding: utf-8 -*-
import numpy as np

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
#N, D_in, H, D_out = 64, 1000, 100, 10
N, D_in, H, D_out = 4, 10, 6, 3

# 创建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

print("x shape:", x.shape)
print("y shape:", y.shape)
print("w1 shape:", w1.shape)
print("w2 shape:", w2.shape)

learning_rate = 1e-6
for t in range(500):
    print('\nforward ','*'*50)
    # 前向传递：计算预测值y
    h = x.dot(w1) # x是N*D_in w1是D_in*H，输出h是 N*H，dot是矩阵乘法
    print("h shape:", h.shape)
    h_relu = np.maximum(h, 0) # 对h的每一个元素v，求maxmium(v, 0),即Relu，输出依然是N*H
    print("h_relu shape:", h_relu.shape)
    y_pred = h_relu.dot(w2) # 输入N*H, w2是H*D_out，输出N*D_out
    print("y_pred shape:", y_pred.shape)

    # 计算和打印损失loss
    loss = np.square(y_pred - y).sum() # y_pred-y是每个元素相减，依然是H*D_out，然后各个元素求square即平方，然后将sum平方误差加起来
    print("y_pred - y shape:", (y_pred - y).shape) # 维度是N*D_out
    print("np.square(y_pred - y) shape:", np.square(y_pred - y).shape) #维度同上，依然是# 维度是N*D_out
    print("loss shape:", loss.shape) #loss是一个标量，即loss是一个数。np里对矩阵的sum()是对矩阵的每个元素求sum,返回的是标量
    print(t, loss)
    
    print('\nbackward ','*'*50)

    # 反向传播，计算w1和w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y) # 维度是N*D_out，和y和y_pred一样
    print("grad_y_pred shape:", grad_y_pred.shape)
    grad_w2 = h_relu.T.dot(grad_y_pred) #w2的梯度的维度和w2一样
    print("grad_w2 shape:", grad_w2.shape)
    grad_h_relu = grad_y_pred.dot(w2.T) #h_relu的梯度的维度和h_relu一样
    print("grad_h_relu shape:", grad_h_relu.shape)
    grad_h = grad_h_relu.copy()
    print("grad_h shape:", grad_h.shape)
    print('1 grad_h:', grad_h)
    print('h:', h)
    grad_h[h < 0] = 0  #这个意思是，在grad_h的每个位置，如果h对应位置上的值小于0，则将grad_h此位置置为0，否则grad_h此位置保持原值
    print('2 grad_h:', grad_h)
    grad_w1 = x.T.dot(grad_h)
    print('grad_w1:', grad_w1)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

x shape: (4, 10)
y shape: (4, 3)
w1 shape: (10, 6)
w2 shape: (6, 3)

forward  **************************************************
h shape: (4, 6)
h_relu shape: (4, 6)
y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred - y) shape: (4, 3)
loss shape: ()
0 352.92012594689965

backward  **************************************************
grad_y_pred shape: (4, 3)
grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.74267724  21.99468139  -9.0196175    8.21079541  -1.19548022
   16.56523062]
 [ -1.38980758 -10.6583527    1.79716781  -4.0991212   12.75616236
    5.62949497]
 [ -4.77369495   0.09549317  -1.83697816   6.85110825  10.82869565
   17.32742655]
 [-21.23694399 -25.0357431    5.14831292  26.58727682  37.7815345
   48.75554909]]
h: [[-1.89595328  3.2730693   1.01109382 -0.72619071 -0.29911145  3.98416582]
 [ 6.37493904 -5.48610974 -0.71747675 -2.84774457  4.93929022 -2.65300541]
 [-2.46336953 -0.97256955  1.30767489 -0.74439711 -0.64701139  2

   47.70346608]]
h: [[-1.89605247  3.26747742  1.01288194 -0.72934626 -0.30268415  3.97324415]
 [ 6.37546307 -5.48452763 -0.71883678 -2.85153675  4.92908618 -2.6589975 ]
 [-2.46335098 -0.97381167  1.30853991 -0.74716715 -0.6511182   2.85500462]
 [-1.598749   -2.12481742  0.18407548  2.91586491  2.83264017  6.11625806]]
2 grad_h: [[ 0.         21.91132835 -9.0066934   0.          0.         16.20472046]
 [-1.36252588  0.          0.          0.         12.53498878  0.        ]
 [ 0.          0.         -1.84638274  0.          0.         17.00083019]
 [ 0.          0.          5.06980202 26.10889095 37.10149728 47.70346608]]
grad_w1: [[ -0.58141343  -0.28430574   8.79634623  37.94527621  59.27024393
   57.04543823]
 [ -0.97485206  24.86699524  -3.27458745  48.17454803  77.42590164
  128.57713601]
 [ -0.54203851   8.36019489   2.99647356  23.82779289  38.84664534
   33.08873291]
 [ -0.52928985  -5.07438056   4.58205168  28.00612493  44.66689205
   74.5058573 ]
 [ -0.54071367 -11.09203115

grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.61296779  21.84331964  -8.99574961   7.94323609  -1.43194527
   15.91867084]
 [ -1.34065577 -10.37936768   1.73701248  -4.15922036  12.35857644
    5.31564124]
 [ -4.67551795   0.10722888  -1.85364779   6.59516142  10.51706729
   16.74184752]
 [-20.8098488  -24.58625903   5.00773615  25.7297658   36.56264057
   46.8709    ]]
h: [[-1.896132    3.26292836  1.0143464  -0.73188033 -0.3055528   3.96450122]
 [ 6.37588322 -5.48324057 -0.71993796 -2.85458208  4.92088965 -2.66379418]
 [-2.4633361  -0.97482214  1.30925482 -0.74939163 -0.65441625  2.84353102]
 [-1.59859019 -2.12695608  0.1821517   2.90062579  2.80952257  6.08543671]]
2 grad_h: [[ 0.         21.84331964 -8.99574961  0.          0.         15.91867084]
 [-1.34065577  0.          0.          0.         12.35857644  0.        ]
 [ 0.          0.         -1.85364779  0.          0.         16.74184752]
 [ 0.          0.          5.00773615 25.7297658 

 [ 0.          0.          4.9459303  25.35136798 36.0248905  46.04103514]]
grad_w1: [[ -0.56267075  -0.28251939   8.6262128   36.84433253  57.55471488
   55.00437111]
 [ -0.94342633  24.71075047  -3.4963049   46.77681241  75.18648953
  124.18583654]
 [ -0.52456513   8.30766596   2.90604399  23.13645366  37.72347001
   31.85966474]
 [ -0.51222745  -5.04249712   4.42111001  27.19355563  43.37472973
   72.03031934]
 [ -0.52328301 -11.02233749   1.40429389 -11.4942147  -11.4993692
  -20.80488481]
 [  2.40091742  18.87782087  -1.25445885  34.16430479  26.36818404
   76.75557794]
 [ -1.06455802  -4.17440586  12.47632699  47.55724171  77.41453373
   70.29986008]
 [ -0.58511399  17.58701131  -8.33378127 -17.77314636 -19.85067338
  -40.82625304]
 [  0.90959999 -21.35074171   7.84232042 -17.08674584 -32.68371256
  -67.321525  ]
 [  1.85049736  22.33430312 -13.80075834 -21.35502499 -47.44121295
  -19.03629733]]

forward  **************************************************
h shape: (4, 6)
h_relu s

grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.49569037  21.70065069  -8.97169238   7.70403278  -1.63923729
   15.34087756]
 [ -1.29586057 -10.12425421   1.68266841  -4.20926238  11.99968862
    5.03766847]
 [ -4.58666239   0.11695477  -1.8677682    6.36637355  10.23942142
   16.21915213]
 [-20.42216541 -24.17857663   4.88300408  24.96519426  35.47616829
   45.19528305]]
h: [[-1.89629487  3.25342133  1.01743448 -0.73708392 -0.31144251  3.94662744]
 [ 6.37674371 -5.48055075 -0.72222488 -2.86083549  4.90405378 -2.67360076]
 [-2.46330563 -0.97693391  1.31078033 -0.7539595  -0.66118877  2.82003188]
 [-1.59826495 -2.13142564  0.17822606  2.8693331   2.76205013  6.02242794]]
2 grad_h: [[ 0.         21.70065069 -8.97169238  0.          0.         15.34087756]
 [-1.29586057  0.          0.          0.         11.99968862  0.        ]
 [ 0.          0.         -1.8677682   0.          0.         16.21915213]
 [ 0.          0.          4.88300408 24.96519426

...

grad_w1: [[ -0.52206534  -0.27845142   8.27037569  34.53515794  53.9521329
   50.74645316]
 [ -0.87534352  24.35494327  -3.94454089  43.84513149  70.48154387
  114.99620974]
 [ -0.48670964   8.18804485   2.71922277  21.68640402  35.36316275
   29.29567148]
 [ -0.47526231  -4.9698908    4.08733537  25.48923197  40.66027207
   66.85220473]
 [ -0.48552004 -10.86362815   1.47937181 -10.77382851 -10.7756716
  -19.14826442]
 [  2.22765405  18.60600134  -1.5552499   32.02309774  24.70135832
   70.95722505]
 [ -0.98773367  -4.11429908  12.00377689  44.57664832  72.57073617
   64.88003699]
 [ -0.54288895  17.33377799  -8.07204099 -16.65923561 -18.60335243
  -37.96728881]
 [  0.84395827 -21.04331488   7.99631348 -16.01585443 -30.64153236
  -62.41868931]
 [  1.71695533  22.0127141  -13.53217882 -20.01662427 -44.48019555
  -17.59096336]]

forward  **************************************************
h shape: (4, 6)
h_relu shape: (4, 6)
y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred -

h shape: (4, 6)
h_relu shape: (4, 6)
y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred - y) shape: (4, 3)
loss shape: ()
360 296.323278626388

backward  **************************************************
grad_y_pred shape: (4, 3)
grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.25126603  21.38414415  -8.91350063   7.21335613  -2.05130181
   14.15650863]
 [ -1.20132039  -9.58318718   1.56944526  -4.29994621  11.25286255
    4.47594809]
 [ -4.40105893   0.13438204  -1.89422094   5.8971572    9.67287281
   15.14948999]
 [-19.60852542 -23.32398546   4.63010577  23.40307312  33.2574007
   41.78678594]]
h: [[-1.89663818  3.23248018  1.02436129 -0.74813091 -0.32394427  3.90905022]
 [ 6.37855743 -5.47462587 -0.72719886 -2.8741112   4.86830103 -2.69422404]
 [-2.46324141 -0.98158553  1.3142825  -0.76365687 -0.67556709  2.77042855]
 [-1.5975794  -2.14127075  0.17000134  2.80290012  2.66126305  5.88995056]]
2 grad_h: [[ 0.         21.38414415 -8.913

y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred - y) shape: (4, 3)
loss shape: ()
404 290.38767620815474

backward  **************************************************
grad_y_pred shape: (4, 3)
grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.19688853  21.30996099  -8.89898741   7.10556834  -2.13928475
   13.89651535]
 [ -1.18005395  -9.46098487   1.54424583  -4.31753161  11.0868134
    4.3542067 ]
 [ -4.35966557   0.13771176  -1.89954551   5.79410034   9.54898239
   14.91499144]
 [-19.42629963 -23.13277992   4.57511291  23.06106768  32.77183733
   41.0434034 ]]
h: [[-1.89671524  3.2275975   1.02599977 -0.75062936 -0.32677183  3.90062369]
 [ 6.37896457 -5.47324442 -0.72834707 -2.87711371  4.86021495 -2.69885082]
 [-2.463227   -0.9826701   1.31512566 -0.76585008 -0.67881904  2.75926573]
 [-1.59742551 -2.14356625  0.16816153  2.78787529  2.63846799  5.8602379 ]]
2 grad_h: [[ 0.         21.30996099 -8.89898741  0.          0.         13.89

grad_w1: [[ -0.4948269   -0.27556379   8.04115811  33.04258681  51.62032056
   48.01222442]
 [ -0.829673    24.10237434  -4.22076933  41.95019365  67.4345225
  109.0730237 ]
 [ -0.46131586   8.10313208   2.60075912  20.74914175  33.83416246
   27.64923098]
 [ -0.45046579  -4.91835136   3.87459872  24.3876157   38.90259251
   63.51641619]
 [ -0.46018833 -10.75096868   1.52447361 -10.30819562 -10.3124103
  -18.08196005]
 [  2.11142756  18.41305087  -1.74121684  30.63909505  23.644068
   67.22256658]
 [ -0.93619927  -4.07163242  11.69832057  42.65009513  69.43303487
   61.40035086]
 [ -0.51456406  17.15402091  -7.90055154 -15.93924197 -17.8023346
  -36.12821233]
 [  0.79992526 -20.82508865   8.08532314 -15.323667   -29.31491803
  -59.25312243]
 [  1.62737423  21.78443488 -13.35204073 -19.15152803 -42.55238684
  -16.66981544]]

forward  **************************************************
h shape: (4, 6)
h_relu shape: (4, 6)
y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred - y)

h shape: (4, 6)
h_relu shape: (4, 6)
y_pred shape: (4, 3)
y_pred - y shape: (4, 3)
np.square(y_pred - y) shape: (4, 3)
loss shape: ()
491 279.1839224181803

backward  **************************************************
grad_y_pred shape: (4, 3)
grad_w2 shape: (6, 3)
grad_h_relu shape: (4, 6)
grad_h shape: (4, 6)
1 grad_h: [[ -0.09266843  21.16367223  -8.86947931   6.90030318  -2.30413333
   13.40160566]
 [ -1.13903793  -9.22479007   1.49591857  -4.34850419  10.76853887
    4.12414625]
 [ -4.2801986    0.14350913  -1.90916486   5.59786108   9.31364226
   14.4689195 ]
 [-19.07561379 -22.76501182   4.47097621  22.41093956  31.84904053
   39.63330174]]
h: [[-1.89686362  3.21799298  1.02924733 -0.75546352 -0.33224341  3.88439791]
 [ 6.3797485  -5.47052701 -0.73059398 -2.88292317  4.84457157 -2.70776314]
 [-2.46319924 -0.98480354  1.31681203 -0.77009365 -0.68511121  2.73772774]
 [-1.59712921 -2.14808164  0.16462269  2.7588042   2.5943624   5.80301542]]
2 grad_h: [[ 0.         21.16367223 -8.8

In [10]:
# 2 PyTorch：张量
# Numpy是一个很棒的框架，但它不能利用GPU来加速其数值计算。 
# 对于现代深度神经网络，GPU通常提供50倍或更高的加速，所以，numpy不能满足当代深度学习的需求。

# 在这里，先介绍最基本的PyTorch概念：
# 张量（Tensor）：PyTorch的tensor在概念上与numpy的array相同： 
# tensor是一个n维数组，PyTorch提供了许多函数用于操作这些张量。
# 任何希望使用NumPy执行的计算也可以使用PyTorch的tensor来完成，可以认为它们是科学计算的通用工具。

# 与Numpy不同，PyTorch可以利用GPU加速其数值计算。要在GPU上运行Tensor,在构造张量使用device参数把tensor建立在GPU上。

# 在这里，本章使用tensors将随机数据上训练一个两层的网络。
# 和前面NumPy的例子类似，我们使用PyTorch的tensor，手动在网络中实现前向传播和反向传播：

# liujia: 我的理解就是tensor就是一种可以运行在GPU上的numpy....

# -*- coding: utf-8 -*-

import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

#创建随机输入和输出数据，注意可以选择device和dtype....
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测y
    h = x.mm(w1) # mm是矩阵乘法
    
    # 将输入input张量每个元素的夹紧到区间 [min,max][min,max]，并返回结果到一个新张量, 这里将min设置为0，即意味着将tensor的所有值如果小于0变为0，否则不变
    # 就是Relu
    h_relu = h.clamp(min=0) 
    y_pred = h_relu.mm(w2)

    # 计算和打印损失
    loss = (y_pred - y).pow(2).sum().item() # pow(2)即所有位置的值平方，然后sum()求所有位置的和，最后item()转为标量
    print(t, loss)

    # Backprop计算w1和w2相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 30614876.0
1 25232606.0
2 21621492.0
3 17618002.0
4 13266533.0
5 9231562.0
6 6128486.0
7 4014908.5
8 2685485.75
9 1869183.625
10 1364514.75
11 1041933.0
12 826141.0
13 674567.4375
14 562733.1875
15 476903.53125
16 408842.375
17 353545.9375
18 307821.90625
19 269523.03125
20 237090.640625
21 209395.984375
22 185588.953125
23 165031.484375
24 147223.703125
25 131726.1875
26 118209.1484375
27 106327.2734375
28 95857.3984375
29 86601.15625
30 78395.6484375
31 71106.40625
32 64613.7421875
33 58814.453125
34 53625.5390625
35 48971.7421875
36 44790.5078125
37 41032.109375
38 37643.84375
39 34583.51171875
40 31813.646484375
41 29302.05078125
42 27020.888671875
43 24947.400390625
44 23058.0546875
45 21334.966796875
46 19762.3515625
47 18324.92578125
48 17009.40234375
49 15805.8505859375
50 14701.0498046875
51 13685.9013671875
52 12752.0009765625
53 11891.5078125
54 11097.720703125
55 10365.0458984375
56 9688.1201171875
57 9062.037109375
58 8482.1796875
59 7944.89453125
60 7446.8427734375
61 6

In [11]:
# 3 PyTorch：定义新的自动求导函数
# 在底层，每一个原始的自动求导运算实际上是两个在Tensor上运行的函数。
# 其中，forward函数计算从输入Tensors获得的输出Tensors。而backward函数接收输出Tensors对于某个标量值(感觉这个标量是loss)的梯度，
# 并且计算输入Tensors相对于该相同标量值的梯度。
# liujia: forward就是根据输入计算输出，backward就是根据“输出的梯度”反向求“输入对于此的梯度”

# 在PyTorch中，我们可以很容易地通过定义torch.autograd.Function的子类并实现forward和backward函数，
# 来定义自己的自动求导运算。之后我们就可以使用这个新的自动梯度运算符了。然后，我们可以通过构造一个实例并像调用函数一样，
# 传入包含输入数据的tensor调用它，这样来使用新的自动求导运算。

# 这个例子中，我们自定义一个自动求导函数来展示ReLU的非线性。并用它实现我们的两层网络：
import torch

class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，
    并完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，
        并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(x) #使用ctx缓存输入对象x,方便反向传播时使用
        return x.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output): # grad_output是输出的梯度
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。 
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors # 从ctx拿到forward时缓存的输入对象
        grad_x = grad_output.clone()
        grad_x[x < 0] = 0
        return grad_x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 正向传播：使用张量上的操作来计算输出值y；
    # 我们通过调用 MyReLU.apply 函数来使用自定义的ReLU
    y_pred = MyReLU.apply(x.mm(w1)).mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播过程。
    loss.backward() #之前参与计算的一些tensor的梯度都自动计算出来了，保存在其 .grad变量里

    # 这里更新参数时，不计算梯度。。。
    with torch.no_grad():
        # 用梯度下降更新权重
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 在反向传播之后手动清零梯度 #liujia: 似乎每一轮迭代都应该清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

0 34886464.0
1 36276724.0
2 42040236.0
3 43657316.0
4 34825428.0
5 20155242.0
6 9100769.0
7 4008884.0
8 2113322.75
9 1390026.625
10 1054688.125
11 856702.0
12 717410.4375
13 609765.0
14 523016.03125
15 451712.125
16 392357.75
17 342459.5625
18 300307.9375
19 264452.5625
20 233751.140625
21 207331.671875
22 184495.90625
23 164669.640625
24 147391.75
25 132266.4375
26 118980.90625
27 107284.84375
28 96945.3359375
29 87778.6796875
30 79638.7890625
31 72386.4921875
32 65894.328125
33 60082.15625
34 54867.41796875
35 50180.77734375
36 45965.8671875
37 42162.44140625
38 38725.09375
39 35615.5546875
40 32797.5
41 30238.013671875
42 27910.392578125
43 25794.740234375
44 23864.28125
45 22100.91015625
46 20488.73828125
47 19015.19140625
48 17664.974609375
49 16425.169921875
50 15285.837890625
51 14237.70703125
52 13273.072265625
53 12383.400390625
54 11562.5615234375
55 10804.275390625
56 10103.076171875
57 9454.2529296875
58 8853.5302734375
59 8296.587890625
60 7780.107421875
61 7300.5654296875

463 0.0003985028015449643
464 0.0003893519169650972
465 0.0003787378373090178
466 0.00037024830817244947
467 0.0003610217245295644
468 0.0003525354550220072
469 0.0003443825407885015
470 0.0003359665279276669
471 0.00032772653503343463
472 0.00032013305462896824
473 0.0003124629147350788
474 0.0003052707761526108
475 0.00029779388569295406
476 0.0002909224422182888
477 0.00028336458490230143
478 0.00027789879823103547
479 0.0002711504348553717
480 0.000264680857071653
481 0.0002594826219137758
482 0.0002533215447328985
483 0.00024805229622870684
484 0.0002421781828161329
485 0.00023693584080319852
486 0.00023177784169092774
487 0.00022722859284840524
488 0.0002217974397353828
489 0.00021813003695569932
490 0.0002132391819031909
491 0.0002083357539959252
492 0.0002038574602920562
493 0.0002000478416448459
494 0.00019622896797955036
495 0.00019202989642508328
496 0.00018759374506771564
497 0.00018323473341297358
498 0.00017994351219385862
499 0.00017592163931112736


In [12]:
# 4 nn模块

# 计算图和autograd是十分强大的工具，可以定义复杂的操作并自动求导；然而对于大规模的网络，autograd太过于底层。 
# 在构建神经网络时，我们经常考虑将计算安排成层，其中一些具有可学习的参数，它们将在学习过程中进行优化。

# TensorFlow里，有类似Keras，TensorFlow-Slim和TFLearn这种封装了底层计算图的高度抽象的接口，这使得构建网络十分方便。

# 在PyTorch中，包nn完成了同样的功能。nn包中定义一组大致等价于层的模块。
# 一个模块接受输入的tesnor，计算输出的tensor，而且还保存了一些内部状态比如需要学习的tensor的参数等。
# nn包中也定义了一组损失函数（loss functions），用来训练神经网络。

# -*- coding: utf-8 -*-
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

#创建输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H), #定义一个线性模块，可以理解为全连接层，两个参数分别是输入的维度和输出的维度
    torch.nn.ReLU(), #Relu层
    torch.nn.Linear(H, D_out),
)

# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum') # 'elementwise_mean' 均方误差更常用。 MSELoss感觉就是平方误差....

#每个epoch大概就是做几项工作
# 1）通过dataloader遍历数据，batch的。。。
# 2）前向计算
# 3）计算误差
# 4）清空梯度，然后做反向传播(计算梯度)
# 5）更新参数

learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)

     # 计算并打印损失。
     # 传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 反向传播之前清零梯度
    model.zero_grad()  #这个不要忘。。。。

    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()

    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 696.8065795898438
1 645.02880859375
2 600.1849365234375
3 561.0460205078125
4 526.4741821289062
5 495.3044128417969
6 467.0234680175781
7 441.069091796875
8 416.93115234375
9 394.5049743652344
10 373.56231689453125
11 353.8485107421875
12 335.2295227050781
13 317.5515441894531
14 300.7132873535156
15 284.6492004394531
16 269.3376770019531
17 254.79930114746094
18 240.9488983154297
19 227.7096710205078
20 215.0552978515625
21 202.9818115234375
22 191.44378662109375
23 180.4709930419922
24 170.02999877929688
25 160.13076782226562
26 150.72140502929688
27 141.7903289794922
28 133.3374786376953
29 125.33795166015625
30 117.79357147216797
31 110.6729507446289
32 103.95609283447266
33 97.60586547851562
34 91.64877319335938
35 86.04741668701172
36 80.77972412109375
37 75.82501220703125
38 71.1649169921875
39 66.7774429321289
40 62.6696891784668
41 58.811431884765625
42 55.193572998046875
43 51.80120086669922
44 48.621543884277344
45 45.61186981201172
46 42.792381286621094
47 40.153232574462

440 2.467583544785157e-05
441 2.4011325876927003e-05
442 2.3365682864096016e-05
443 2.2737584004062228e-05
444 2.2126636395114474e-05
445 2.1532892787945457e-05
446 2.0955432773916982e-05
447 2.0393872546264902e-05
448 1.984737355087418e-05
449 1.9315635654493235e-05
450 1.87977548193885e-05
451 1.8295189875061624e-05
452 1.7804952221922576e-05
453 1.7328100511804223e-05
454 1.6863905329955742e-05
455 1.6413752746302634e-05
456 1.597547270648647e-05
457 1.5549185263807885e-05
458 1.5133890883589629e-05
459 1.4729387658007909e-05
460 1.4336276763060596e-05
461 1.3953961570223328e-05
462 1.3581196981249377e-05
463 1.3219995707913768e-05
464 1.2867928489868063e-05
465 1.252474157809047e-05
466 1.2192450412840117e-05
467 1.1867850844282657e-05
468 1.1552430805750191e-05
469 1.124551818065811e-05
470 1.0945852409349754e-05
471 1.0655440746631939e-05
472 1.037277343129972e-05
473 1.0096749974763952e-05
474 9.829669579630718e-06
475 9.568361747369636e-06
476 9.314697308582254e-06
477 9.068115

In [13]:
# optim模块，封装了各种优化算法，用于更新参数

# 到目前为止，我们已经通过手动改变包含可学习参数的张量来更新模型的权重。
# 对于随机梯度下降(SGD/stochastic gradient descent)等简单的优化算法来说，这不是一个很大的负担，
# 但在实践中，我们经常使用AdaGrad、RMSProp、Adam等更复杂的优化器来训练神经网络。

import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包定义模型和损失函数
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用optim包定义优化器（Optimizer）。Optimizer将会为我们更新模型的权重。
# 这里我们使用Adam优化方法；optim包还包含了许多别的优化算法。
# Adam构造函数的第一个参数告诉优化器应该更新哪些张量。
learning_rate = 1e-4
#liujia: Adam优化，关联了modle的参数(反向传播后就能更新这些关联了的参数了)，指定了learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 同上，每个epoch大概就是做几项工作
# 1）通过dataloader遍历数据，batch的。。。
# 2）前向计算
# 3）计算误差
# 4）optimizer清空梯度，然后loss来做反向传播(计算梯度)
# 5）optimizer更新参数
for t in range(500):

    # 前向传播：通过像模型输入x计算预测的y
    y_pred = model(x)

    # 计算并打印loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)
    optimizer.zero_grad()

    # 反向传播：根据模型的参数计算loss的梯度
    loss.backward()

    # 调用Optimizer的step函数使它所有参数更新
    optimizer.step()

0 662.63037109375
1 645.2057495117188
2 628.3567504882812
3 611.9851684570312
4 596.1473999023438
5 580.8091430664062
6 565.9033203125
7 551.3787841796875
8 537.2445678710938
9 523.442138671875
10 510.01763916015625
11 496.95904541015625
12 484.2630920410156
13 471.9414367675781
14 460.0780029296875
15 448.57269287109375
16 437.379150390625
17 426.4809265136719
18 415.9138488769531
19 405.6692810058594
20 395.71209716796875
21 386.04205322265625
22 376.6095886230469
23 367.42218017578125
24 358.51470947265625
25 349.846923828125
26 341.3879089355469
27 333.1212158203125
28 325.0613098144531
29 317.1928405761719
30 309.5125427246094
31 302.0479431152344
32 294.7503967285156
33 287.6220397949219
34 280.6665344238281
35 273.8905334472656
36 267.2750549316406
37 260.81353759765625
38 254.52108764648438
39 248.373046875
40 242.35369873046875
41 236.4476776123047
42 230.6815185546875
43 225.02610778808594
44 219.4758758544922
45 214.04486083984375
46 208.72500610351562
47 203.5265350341797
4

452 1.5014274268310146e-08
453 1.3751199290368277e-08
454 1.2604690624584691e-08
455 1.1552910628154223e-08
456 1.0572750674953113e-08
457 9.687590285523129e-09
458 8.858638267383867e-09
459 8.112081673061766e-09
460 7.427125137127177e-09
461 6.8084800020074e-09
462 6.22967766261695e-09
463 5.692660565159713e-09
464 5.189666918425928e-09
465 4.769522554681771e-09
466 4.357101346386116e-09
467 4.007272735861989e-09
468 3.6552854076887797e-09
469 3.353323174692946e-09
470 3.090904865388211e-09
471 2.820770506062331e-09
472 2.5881534693183994e-09
473 2.3888124811577427e-09
474 2.1947961226231882e-09
475 2.0208081874528716e-09
476 1.854676301427105e-09
477 1.711772279477941e-09
478 1.573884356176336e-09
479 1.457414966488102e-09
480 1.3493993700208762e-09
481 1.2433098994790726e-09
482 1.152809847582148e-09
483 1.0713701037445844e-09
484 9.92434023849853e-10
485 9.186775784542078e-10
486 8.592203615265248e-10
487 7.915978428307824e-10
488 7.389098777288439e-10
489 6.929087859042227e-10
490

In [14]:
# 自定义nn模块

# 有时候需要指定比现有模块序列更复杂的模型；对于这些情况，可以通过继承nn.Module并定义forward函数，
# 这个forward函数可以 使用其他模块或者其他的自动求导运算来接收输入tensor，产生输出tensor。

# 在这个例子中，我们用自定义Module的子类构建两层网络：
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。
        """
        super(TwoLayerNet, self).__init__() # 记得调用父类的初始化函数
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意的（可微分的）操作。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 通过实例化上面定义的类来构建我们的模型。
model = TwoLayerNet(D_in, H, D_out)

# 构造损失函数和优化器。
# SGD构造函数中对model.parameters()的调用，
# 将包含模型的一部分，即两个nn.Linear模块的可学习参数。
loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4) #这次是用标准的随机梯度下降

for t in range(500):
    # 前向传播：通过向模型传递x计算预测值y
    y_pred = model(x)

    #计算并输出loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 684.0811157226562
1 634.2269287109375
2 591.024169921875
3 553.3485717773438
4 519.8214111328125
5 489.459716796875
6 461.8538818359375
7 436.5088195800781
8 412.8912048339844
9 390.91461181640625
10 370.4383850097656
11 351.0810546875
12 332.743896484375
13 315.2957763671875
14 298.66900634765625
15 282.9261169433594
16 267.90216064453125
17 253.57025146484375
18 239.8653106689453
19 226.7836456298828
20 214.30079650878906
21 202.3874053955078
22 191.0276336669922
23 180.20448303222656
24 169.89796447753906
25 160.0966796875
26 150.7695770263672
27 141.88897705078125
28 133.48211669921875
29 125.49744415283203
30 117.93720245361328
31 110.79247283935547
32 104.05890655517578
33 97.6992416381836
34 91.71428680419922
35 86.08689880371094
36 80.79637908935547
37 75.81285095214844
38 71.13111877441406
39 66.7333984375
40 62.60694122314453
41 58.7327995300293
42 55.091556549072266
43 51.667232513427734
44 48.45404052734375
45 45.445579528808594
46 42.624568939208984
47 39.9826774597168
4

485 1.7590764400665648e-05
486 1.7196181943290867e-05
487 1.680966670392081e-05
488 1.6431487892987207e-05
489 1.60625932039693e-05
490 1.5702282325946726e-05
491 1.5349976820289157e-05
492 1.5006233297754079e-05
493 1.4669477423012722e-05
494 1.4340883353725076e-05
495 1.4019402442499995e-05
496 1.3705576748179737e-05
497 1.3397784641711041e-05
498 1.309927483816864e-05
499 1.2805474398192018e-05


In [ ]:
# 控制流和权重共享

# 作为动态图和权重共享的一个例子，我们实现了一个非常奇怪的模型：一个全连接的ReLU网络，在每一次前向传播时，
# 它的隐藏层的层数为随机1到4之间的数，这样可以多次重用相同的权重来计算。

# 因为这个模型可以使用普通的Python流控制来实现循环，并且我们可以通过在定义转发时多次重用同一个模块来实现最内层之间的权重共享。

# 我们利用Mudule的子类很容易实现这个模型：

import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们构造了三个nn.Linear实例，它们将在前向传播时被使用。
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        对于模型的前向传播，我们随机选择0、1、2、3，
        并重用了多次计算隐藏层的middle_linear模块。
        由于每个前向传播构建一个动态计算图，
        我们可以在定义模型的前向传播时使用常规Python控制流运算符，如循环或条件语句。
        在这里，我们还看到，在定义计算图形时多次重用同一个模块是完全安全的。
        这是Lua Torch的一大改进，因为Lua Torch中每个模块只能使用一次。
        """
        #liujia: 这里要理解的有两个地方
        # 动态计算图：即模型是可以动态修改的，在每次前向计算时，都会构建动态图，使得每次前向时的计算图可以不一样！！！
        # 模块可以重用多次！！！是安全的！！！
        # 问题来了，假如这里随机数是2，即重用了两次，那么更新参数时，是更新一次还是更新两次？
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 实例化上面定义的类来构造我们的模型
model = DynamicNet(D_in, H, D_out)

# 构造我们的损失函数（loss function）和优化器（Optimizer）。
# 用平凡的随机梯度下降训练这个奇怪的模型是困难的，所以我们使用了momentum方法。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):

    # 前向传播：通过向模型传入x计算预测的y。
    y_pred = model(x)

    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()